In [1]:
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import MACCSkeys
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols

import urllib
from urllib.request import urlopen
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#hiding warning messages
import warnings
warnings.filterwarnings("ignore")

#Reading in Molecular Properties CSV
data = pd.read_csv('Downsampled_NCGC_Full')
#data = data.astype(float, errors = 'ignore')
data.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,Activity Summary,PUBCHEM_CID,...,RotatableBondCount,CanonicalSMILES,IsomericSMILES,InChI,XLogP,ExactMass,TPSA,HeavyAtomCount,Complexity,target
0,-1.755092,-1.454071,0.054049,0.639525,-0.149290,0.287549,-0.182050,0.012485,active antagonist,8247.0,...,2.0,CC(=O)NNC1=CC=CC=C1,CC(=O)NNC1=CC=CC=C1,InChI=1S/C8H10N2O/c1-7(11)9-10-8-5-3-2-4-6-8/h...,0.7,150.079313,41.1,11.0,130.0,1
1,-2.047865,-0.160582,-0.757108,0.508870,0.258721,0.095206,0.261830,0.090075,active antagonist,13636.0,...,0.0,C1=C(C(=C(C(=C1Cl)Cl)O)Cl)Cl,C1=C(C(=C(C(=C1Cl)Cl)O)Cl)Cl,InChI=1S/C6H2Cl4O/c7-2-1-3(8)5(10)6(11)4(2)9/h...,3.9,231.883025,20.2,11.0,129.0,1
2,1.976719,-0.285250,-0.244699,0.971285,0.405076,-0.256409,0.040242,-0.010849,active antagonist,442530.0,...,5.0,CC1CC2=C(C=C(C(=C2C(=O)O1)O)C(=O)NC(CC3=CC=CC=...,C[C@@H]1CC2=C(C=C(C(=C2C(=O)O1)O)C(=O)N[C@@H](...,InChI=1S/C20H18ClNO6/c1-10-7-12-14(21)9-13(17(...,4.7,403.082265,113.0,28.0,608.0,1
3,2.437582,1.988528,0.605595,1.034114,-0.011421,-0.418295,0.217261,-0.174805,active antagonist,3392731.0,...,12.0,CC(C)(C(=O)O)SC1=CC=C(C=C1)CCN(CCCCC2CCCCC2)C(...,CC(C)(C(=O)O)SC1=CC=C(C=C1)CCN(CCCCC2CCCCC2)C(...,"InChI=1S/C29H46N2O3S/c1-29(2,27(32)33)35-26-18...",8.0,502.322915,94.9,35.0,636.0,1
4,1.298015,0.251638,-0.315783,1.425453,0.267422,-0.020243,0.006769,-0.274519,active antagonist,5040063.0,...,5.0,CCCC1=C(N(N=C1C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)C4...,CCCC1=C(N(N=C1C2=CC=C(C=C2)O)C3=CC=C(C=C3)O)C4...,InChI=1S/C24H22N2O3/c1-2-3-22-23(16-4-10-19(27...,5.5,386.163043,78.5,29.0,494.0,1


# MACCS Keys

In [2]:
#Making list of isomericSMILES identifiers
iSMILES = data['IsomericSMILES']
iSMILES[0]

'CC(=O)NNC1=CC=CC=C1'

In [3]:
#Making list of molecules from iSMILES
i = 0
ms = []
while (i < len(iSMILES)):
    molecule = Chem.MolFromSmiles(iSMILES[i])
    ms.append(molecule)
    i = i + 1

In [4]:
#Getting MACCS fingerprints list
#Turning fingerprints to bit strings
maccs_fps = [MACCSkeys.GenMACCSKeys(x).ToBitString()[1:] for x in ms ]

# Topological Fingerprint (Daylight Analogue)

In [5]:
from rdkit.Chem import rdmolops

#getting list of topological fingerprints
top_fps = [rdmolops.RDKFingerprint(x, fpSize=2048, minPath=1, maxPath=7).ToBitString() for x in ms]

# Morgan Fingerprint (ECFP)

In [6]:
from rdkit.Chem import AllChem

#getting morgan ecfp fingerprint
ecfp_fps = [AllChem.GetMorganFingerprintAsBitVect(x,4,nBits=1024).ToBitString() for x in ms]

# Morgan Fingerprint (FCFP)

In [7]:
fcfp_fps = [AllChem.GetMorganFingerprintAsBitVect(x,4,nBits=1024,useFeatures=True).ToBitString() for x in ms]

# PubChem FP Decoding

In [12]:
#Getting CID list
CID_list = data['PUBCHEM_CID']

#Making them integers (removing decimals)
CID_list = [int(i) for i in CID_list]

#Changing list to str, dropping start/end brackets, removing spaces
CID_str = (str(list1)[1:-1])
CID_str = CID_str.replace(' ', '')

In [13]:
#Getting the Pubchem Fingerprints for each CID

#opening and reading the 1URLs
url1 = ('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/' + CID_str + '/property/Fingerprint2D/TXT')
html1 = urlopen(url1) 
soup1 = BeautifulSoup(html1, 'lxml')

pub_fp = soup1.get_text()

#pub_fp string to pub_fp list
pub_fp = pub_fp.split()

In [14]:
#Decoding Fingerprints
from base64 import b64decode

def PCFP_BitString(pcfp_base64) :

    pcfp_bitstring = "".join( ["{:08b}".format(x) for x in b64decode( pcfp_base64 )] )[32:913]
    return pcfp_bitstring

i = 0
pub_fp_decoded = []
while (i < len(pub_fp)):
    fp = PCFP_BitString(pub_fp[i])
    pub_fp_decoded.append(fp)
    i = i + 1

# FinalDFTox21

In [18]:
fps_df = pd.DataFrame()
fps_df['Name'] = data['PUBCHEM_CID']
fps_df['MACCS'] = maccs_fps
fps_df['Topological'] = top_fps
fps_df['Morgan ECFP'] = ecfp_fps
fps_df['Morgan FCFP'] = fcfp_fps
fps_df['Pubchem FP'] = pub_fp_decoded
fps_df['Activity'] = data['target']
fps_df.head()

,Name,MACCS,Topological,Morgan ECFP,Morgan FCFP,Pubchem FP,Activity
0,8247.0,0000000000000000000000000000000000000000000000...,1001000000000000000000000000000000000000000000...,0000000000000000000000000000000001000000000010...,1111100000000000000000000000000000000000000000...,1100000001110011001000000000000000000000000000...,1
1,13636.0,0000000000000000000000000000000000000000000000...,0000000000000000000010000010000000000000100000...,0000000000000000000000000000000000000000000000...,0001100010000000000000000000000000000000000000...,0000000001100000001000000000000000000111000000...,1
2,442530.0,0000000000000000000000000000000000000000000000...,1110011011100000111011111110000101001111011100...,0100000000000000000000000000000011000000001000...,1111100010000000100000001100000010000000000000...,1110000001111010001110000000000000000100000000...,1
3,3392731.0,0000000000000000000000000000000000001000000000...,0000000010000100000001000001110010100000001110...,0010100000000000000001000000010101000000000000...,1110100000000000001011001100000010000000000000...,1111000001111011001100000000000001000000000000...,1
4,5040063.0,0000000000000000000000000000000000000000000000...,1100001011111000000110110011010111001001101101...,0000000000000000000000000000000001000101000000...,1001101000000000000000000000000000000010000000...,1110000001111011001100000000000000000000000000...,1


In [19]:
#Writing to csv
fps_df.to_csv('Fingerprints NCGC', index = False)
